In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Data Preparation
train_dir = "../../data4/train"
validation_dir = "../../data4/validation"
batch_size = 32
image_size = (299, 299)

# Augment training data and rescale
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=30,       # Random rotation
    zoom_range=0.3,          # Random zoom
    horizontal_flip=True,    # Random horizontal flip
    brightness_range=[0.8, 1.2],  # Random brightness
    width_shift_range=0.2,   # Horizontal shift
    height_shift_range=0.2,  # Vertical shift
    shear_range=0.2,         # Shearing
    fill_mode="nearest"      # Fill empty pixels
)

# Only rescale validation data
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'  # Changed to 'categorical' for multi-class classification
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Extract number of classes
num_classes = train_generator.num_classes


Found 5999 images belonging to 4 classes.
Found 1672 images belonging to 4 classes.


In [3]:

# Build InceptionV3 Model
def build_inceptionv3_model(num_classes):
    inception_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    inception_base.trainable = False  # Freeze base model

    input_tensor = Input(shape=(299, 299, 3))
    x = inception_base(input_tensor)
    x = GlobalAveragePooling2D()(x)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=output_tensor)
    return model

model = build_inceptionv3_model(num_classes)
model.compile(optimizer=Adam(learning_rate=1e-3), loss='categorical_crossentropy', metrics=['accuracy', 'AUC'])


In [ ]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Training the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/50


/Users/liraj/.pyenv/versions/3.9.4/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 18/188 ━━━━━━━━━━━━━━━━━━━━ 12:48 5s/step - AUC: 0.7199 - accuracy: 0.4395 - loss: 1.8291

In [ ]:
# Evaluate the model
val_loss, val_accuracy, val_auc = model.evaluate(validation_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")
print(f"Validation AUC: {val_auc}")


In [ ]:
# Save the trained model
model.save("../../models/Improved_inceptionV3_model.keras")

In [ ]:
# Classification Report and Confusion Matrix
validation_generator.reset()
predictions = model.predict(validation_generator)
y_pred = np.argmax(predictions, axis=1)
y_true = validation_generator.classes

print("Classification Report")
print(classification_report(y_true, y_pred, target_names=list(validation_generator.class_indices.keys())))

print("Confusion Matrix")
print(confusion_matrix(y_true, y_pred))

In [ ]:
# Fine-Tune the Model
def fine_tune_model(model, base_model, num_trainable_layers):
    """
    Fine-tune the model by unfreezing the top layers of the base model.
    Args:
        model: The compiled model.
        base_model: The pre-trained base model (e.g., InceptionV3).
        num_trainable_layers: Number of layers to unfreeze at the top of the base model.
    Returns:
        A model ready for fine-tuning.
    """
    # Unfreeze the top `num_trainable_layers` layers of the base model
    for layer in base_model.layers[-num_trainable_layers:]:
        layer.trainable = True
    
    # Compile the model with a lower learning rate
    model.compile(
        optimizer=Adam(learning_rate=1e-4),  # Smaller learning rate
        loss='categorical_crossentropy',
        metrics=['accuracy', 'AUC']
    )
    return model

# Fine-tune 30 layers of the InceptionV3 base
num_trainable_layers = 30
model = fine_tune_model(model, model.layers[1], num_trainable_layers)

# Continue training
fine_tune_history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,  # Shorter training for fine-tuning
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the fine-tuned model
val_loss, val_accuracy, val_auc = model.evaluate(validation_generator)
print(f"Validation Loss (Fine-tuned): {val_loss}")
print(f"Validation Accuracy (Fine-tuned): {val_accuracy}")
print(f"Validation AUC (Fine-tuned): {val_auc}")
